# Warehouse Management

In [1]:
from pymongo.collection import Collection
import pymongo

mongodb_uri = "mongodb://192.168.8.10"
db_client = pymongo.MongoClient(mongodb_uri, 27017, serverSelectionTimeoutMS=10000)
db_name = "odoo_data"
alias = "odoo-api-prod"
product_collection = db_client[db_name]['product.product']
product_template_collection = db_client[db_name]['product.template']
partner_collection = db_client[db_name]['res.partner']
stock_location_collection = db_client[db_name]['stock.location']
putaway_rule_collection = db_client[db_name]['stock.putaway.rule']
stock_quant_collection = db_client[db_name]['stock.quant']

## Move Stock based on Putaway Rules

暂存区上架规则:
WH/Stock -> Putaway -> C02-XX-X


In [2]:
import pandas as pd

putaway_rules = putaway_rule_collection.find({"alias": alias})
putaway_rules = list(putaway_rules)
print(f"Alias: {alias}")

list_putaway_rules = []
for item in putaway_rules:
    rule = item['data']
    rule_dict = dict(
        id = rule['id'],
        product_id = rule['product_id'][0],
        product_name = rule['product_id'][1],
        location_in_id = rule['location_in_id'][0],
        location_in_name = rule['location_in_id'][1],
        location_out_id = rule['location_out_id'][0],
        location_out_name = rule['location_out_id'][1],
        active = rule['active'],    
    )
    if rule['active']:
        list_putaway_rules.append(rule_dict)

# 所有上架规则
df_putaway_rules = pd.DataFrame(list_putaway_rules)
print(f"所有上架规则：{len(df_putaway_rules)}")
# 暂存区上架规则
df_putaway_rules_from_staging = df_putaway_rules[df_putaway_rules['location_in_id'] == 8]
df_putaway_rules_from_staging["rule_key"] = df_putaway_rules_from_staging.apply(lambda x: f"{x['product_id']}_{x['location_in_id']}_{x['location_out_id']}", axis=1)
print(f"暂存区上架规则：{len(df_putaway_rules_from_staging)}")
# Remove duplicated according to the rule_key
df_putaway_rules_from_staging = df_putaway_rules_from_staging.drop_duplicates(subset=['rule_key'])
print(f"去重暂存区上架规则：{len(df_putaway_rules_from_staging)}")
df_putaway_rules_from_staging.head()

Alias: odoo-api-prod
所有上架规则：88
暂存区上架规则：88
去重暂存区上架规则：87


,id,product_id,product_name,location_in_id,location_in_name,location_out_id,location_out_name,active,rule_key
0,67,1329,"[DN-211123] Servietten Tissue, 3-lagig, 33 x 3...",8,WH/Stock,429,WH/Stock/[C03] Deichstraße,True,1329_8_429
1,68,1328,"[DN-307888] Serviette 2-lagig 33 x 33 cm, weiß",8,WH/Stock,429,WH/Stock/[C03] Deichstraße,True,1328_8_429
2,69,1336,[DSM-00137050] THERMO ALKA CLEAR,8,WH/Stock,435,WH/Stock/[C00] Rathausmarkt/C00-04-1,True,1336_8_435
3,53,1696,[FBA-HMD-77585] Stretch FIxiervlies elastisch ...,8,WH/Stock,528,WH/Stock/[C02] Mönckebergstraße/C02-07-2,True,1696_8_528
4,54,1697,[FBA-HMD-77590] Stretch FIxiervlies elastisch ...,8,WH/Stock,511,WH/Stock/[C02] Mönckebergstraße/C02-03-1,True,1697_8_511


In [8]:
def query_quant_by_product_and_location(product_id, location_id):
    quant = stock_quant_collection.find_one({"data.product_id": product_id, 
                                             "data.location_id": location_id, })                                             
    return quant

def query_quants_by_product(product_id):
    quants = stock_quant_collection.find({"data.product_id": product_id})
    if quants:
        return list(quants)
    else:
        return []

count = 1
stock_to_move = []
for item in df_putaway_rules_from_staging.to_dict('records'):
    result_quant = query_quant_by_product_and_location(item['product_id'], item['location_in_id'])
    if result_quant:
        quant = result_quant['data']
        quant_id = quant['id']
        quantity = quant['quantity']        
        count += 1
        item['quant_id'] = quant_id
        item['quantity'] = quantity
        stock_to_move.append(item)        

df_stock_to_move = pd.DataFrame.from_dict(stock_to_move)
# Remove rows with quantity = 0
df_stock_to_move = df_stock_to_move[df_stock_to_move['quantity'] > 0]

print(f"需要移库：{len(df_stock_to_move)}")
df_stock_to_move.head()


需要移库：7


,id,product_id,product_name,location_in_id,location_in_name,location_out_id,location_out_name,active,rule_key,quant_id,quantity
1,105,1764,"[DDE-250082] Auslaufhahn, Gewinde 51",8,WH/Stock,523,WH/Stock/[C02] Mönckebergstraße/C02-06-1,True,1764_8_523,3527,10.0
2,106,1765,"[DDE-250089] Kanisterschlüssel schwarz, Gewind...",8,WH/Stock,523,WH/Stock/[C02] Mönckebergstraße/C02-06-1,True,1765_8_523,3529,10.0
3,109,1330,"[DDE-600917] Spray In QF 1L, inkl. 1 Sprühkopf...",8,WH/Stock,532,WH/Stock/[C02] Mönckebergstraße/C02-08-2,True,1330_8_532,3422,24.0
4,103,1332,[DDE-601102] Spray In QF Bio-wipes 70 Tücher,8,WH/Stock,428,WH/Stock/[C02] Mönckebergstraße,True,1332_8_428,3162,15.0
5,107,1738,[DDE-601121] OpSept Classic Händedesinfektion ...,8,WH/Stock,508,WH/Stock/[C02] Mönckebergstraße/C02-02-2,True,1738_8_508,3426,72.0


In [4]:
# 开始移库



## Stock with Locations not Matching the Putaway Rules

检查库存中库位与放置规则不匹配的情况。


In [35]:
for item in df_putaway_rules.to_dict(orient='records'):    
    expect_quant = query_quant_by_product_and_location(item['product_id'], item['location_out_id'])
    actual_quants = query_quants_by_product(item['product_id'])    
    if len(actual_quants) == 0:        
        continue

    if not expect_quant:        
        print(f"Product not found in the expected location [{item['location_out_name']}]")    
        print(f"  * Name: {item['product_name']}")
        for q in actual_quants:
            q = q['data']            
            print(f"  - Expected Location: [{item['location_out_name']}]  ")
            print(f"  - Actual Location: [{q['location_id'][1]}]" )
            print(f"  - Quantity: {q['quantity']}")            
        print()            



Product not found in the expected location [WH/Stock/[B02] Jungfernstieg]
  * Name: [DDE-601107] 1L Spray In QF Flächendesinfektion
  - Expected Location: [WH/Stock/[B02] Jungfernstieg]  
  - Actual Location: [WH/Stock/[C02] Mönckebergstraße]
  - Quantity: 8.0

